In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
df_land = pd.read_csv('C:/Users/benja/OneDrive - University of Tennessee/DATA_499/data/Land_Cover.csv')

df_socioecon = pd.read_csv('C:/Users/benja/OneDrive - University of Tennessee/DATA_499/data/usa_00008(3).csv')

df_politics = pd.read_csv('C:/Users/benja/OneDrive - University of Tennessee/DATA_499/data/countypres_2000-2020.csv')

df_religion = pd.read_excel('C:/Users/benja/OneDrive - University of Tennessee/DATA_499/data/2020_USRC_Summaries.xlsx', sheet_name='2020 Metro Summary')

# Walking score may need different data

# Transit quality:

folder_path = 'C:/Users/benja/OneDrive - University of Tennessee/DATA_499/data/Transit_Qual'
dfs = []

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):  # or '.xls' if your files are in the older Excel format
        # Construct the full file path
        file_path = os.path.join(folder_path, filename)
        
        # Read the Excel file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all the DataFrames in the list into a single DataFrame
df_transit = pd.concat(dfs, ignore_index=True)

df_marijuana = pd.read_excel('C:/Users/benja/OneDrive - University of Tennessee/DATA_499/data/marijuana.xlsx')

cbsa_to_fips = pd.read_csv('C:/Users/benja/OneDrive - University of Tennessee/DATA_499/data/cbsa2fipsxw.csv')

In [3]:
df_land.head(2)

,FIPS,State,County,Year,Total Accounted,Major Crops,Minor Crops,Pasture,Hay,Forest,Developed,Developed Open,Developed Low,Developed Med,Developed High
0,1001,Alabama,Autauga County,2017,379685.68,11944.59,1850.54,20072.88,33499.73,289383.06,22934.88,15903.21,4924.93,1663.29,443.45
1,1001,Alabama,Autauga County,2010,379826.90,12540.83,1492.27,53223.45,21882.95,271317.95,19369.45,13842.28,4108.52,1116.20,302.46


In [4]:
# Ensure FIPS is a string
df_land['FIPS'] = df_land['FIPS'].astype(str)

# Add leading zeros to make FIPS a consistent five digits
df_land['FIPS'] = df_land['FIPS'].apply(lambda x: x.zfill(5))

In [5]:
df_socioecon.head(2)

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,COUNTYFIP,CITYPOP,STRATA,...,SEX,AGE,RACE,RACED,RACHSING,EDUC,EDUCD,EMPSTAT,EMPSTATD,IND
0,2021,202101,1,2021010000026,13.0,2021000000011,1,0,0,80001,...,1,85,1,100,1.0,7,71,3,30,0
1,2021,202101,2,2021010000031,51.0,2021000000021,1,0,0,80001,...,2,67,2,200,2.0,6,63,3,30,0


In [6]:
df_politics.head(2)

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,AL GORE,DEMOCRAT,4942,17208,20220315,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993,17208,20220315,TOTAL


In [7]:
df_religion.head(2)

,CBSA Code,Metro Name,2020 Population,Congregations,Adherents,"Congregations per 100,000 Population",Adherents as % of Population,Population Rank,Congregations Rank,Adherents Rank,"Congregations Per 100,000 Pop. Rank",Adherents as % of Population Rank
0,10100,"Aberdeen, SD Micro Area",42287.0,101.0,31367.0,238.844089,0.741765,672.0,553.0,510.0,158.0,41.0
1,10140,"Aberdeen, WA Micro Area",75636.0,92.0,21494.0,121.635200,0.284177,463.0,597.0,648.0,639.0,879.0


In [8]:
df_transit.head(2)

,place,name,blkgrps,population,households,alltransit_performance_score
0,"""0100100""","""Abanda""",0.03,40.96,15.98,0.0
1,"""0100124""","""Abbeville""",1.34,1282.40,476.84,0.0


In [9]:
df_marijuana.head(2)

,State,Legal Status,Medicinal,Decriminalized
0,Alabama,Mixed,Yes,No
1,Alaska,Fully Legal,Yes,Yes


In [10]:
# 1. df_land should be year = 2022 only
# 2. df_socioecon should be Year = 2022 only and drop columns
# 3. df_socioecon needs FIPS created, with leading zeros restored in STATEFIP and COUNTYFIP and then combined
# 4. does population work?
# 5. df_politics needs county_fips to have decimal dropped
# 6. df_religion needs cbsa converted to fips using cbsa_to_fips
# 7. df_transit needs place renamed FIPS, with the "" removed
# 8. df_marijuana needs STATEFIPS made based on State

In [11]:
df_land_2022 = df_land[df_land['Year'] == 2022]  #1
df_land_2022 = df_land_2022.rename(columns={'Year': 'YEAR'})

In [12]:
df_socioecon_2022 = df_socioecon[df_socioecon['YEAR'] == 2022]  #2
# df_socioecon_2022 = df_socioecon_2022.drop(columns={'RACHSING'})

In [13]:
# Ensure the STATEFIP and COUNTYFIP columns are strings with leading zeros as needed  #3
# We assume 2 characters for state FIPS and 3 for county FIPS as a common case
df_socioecon_2022['STATEFIP_str'] = df_socioecon_2022['STATEFIP'].apply(lambda x: str(x).zfill(2))
df_socioecon_2022['COUNTYFIP_str'] = df_socioecon_2022['COUNTYFIP'].apply(lambda x: str(x).zfill(3))

# Combine the formatted STATEFIP and COUNTYFIP columns to create the FIPS code
df_socioecon_2022['FIPS'] = df_socioecon_2022['STATEFIP_str'] + df_socioecon_2022['COUNTYFIP_str']

# Optionally, you can drop the intermediate string columns if they are no longer needed
df_socioecon_2022.drop(columns=['STATEFIP_str', 'COUNTYFIP_str'], inplace=True)

df_socioecon_2022.head(2)

C:\Users\benja\AppData\Local\Temp\ipykernel_20880\2547242058.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_socioecon_2022['STATEFIP_str'] = df_socioecon_2022['STATEFIP'].apply(lambda x: str(x).zfill(2))
C:\Users\benja\AppData\Local\Temp\ipykernel_20880\2547242058.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_socioecon_2022['COUNTYFIP_str'] = df_socioecon_2022['COUNTYFIP'].apply(lambda x: str(x).zfill(3))
C:\Users\benja\AppData\Local\Temp\ipykernel_20880\2547242058.py:7: SettingWithCopyWarn

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,COUNTYFIP,CITYPOP,STRATA,...,AGE,RACE,RACED,RACHSING,EDUC,EDUCD,EMPSTAT,EMPSTATD,IND,FIPS
3252599,2022,202201,1,2022010000031,69.0,2022000000011,1,97,0,280301,...,85,8,801,NaN,7,71,3,30,0,01097
3252600,2022,202201,2,2022010000111,22.0,2022000000021,1,0,0,200001,...,51,1,100,NaN,6,64,3,30,5391,01000


In [14]:
df_socioecon_2022['CITYPOP'].describe()  #4

count    3.373378e+06
mean     2.638443e+03
std      1.244737e+04
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      8.335900e+04
Name: CITYPOP, dtype: float64

In [15]:
# Filter out zero values in the CITYPOP column and then find the minimum
min_citypop_excluding_zeros = df_socioecon_2022[df_socioecon_2022['CITYPOP'] != 0]['CITYPOP'].min()

print(min_citypop_excluding_zeros)

# This has population of all cities but counties should be added too:
df_countypop = pd.read_csv('C:/Users/benja/OneDrive - University of Tennessee/DATA_499/data/COUNTY_POP.csv')

1008


In [16]:
# Ensure the FIPS column is a string and has leading zeros to make it 5 digits long
df_countypop['FIPS'] = df_countypop['FIPS'].apply(lambda x: str(x).zfill(5))

df_countypop['COUNTY_POP'].describe()

count    2.991000e+03
mean     1.064829e+05
std      3.385943e+05
min      5.100000e+01
25%      1.077000e+04
50%      2.594000e+04
75%      6.799350e+04
max      9.721138e+06
Name: COUNTY_POP, dtype: float64

In [17]:
# Ensure county_fips is treated as a string, and replace the decimal point  #5
df_politics['county_fips'] = df_politics['county_fips'].astype(str).str.replace('.', '')

# If the operation may result in strings longer than 5 digits, truncate to the first 5 digits
df_politics['county_fips'] = df_politics['county_fips'].str[:5]

In [18]:
# Ensure both 'CBSA Code' in df_religion and 'cbsacode' in cbsa_to_fips are strings before merging  #6
df_religion['CBSA Code'] = df_religion['CBSA Code'].astype(str)
cbsa_to_fips['cbsacode'] = cbsa_to_fips['cbsacode'].astype(str)

# Now perform the merge
df_religion = pd.merge(df_religion, cbsa_to_fips, left_on='CBSA Code', right_on='cbsacode', how='left')

# Function to safely convert codes to string with leading zeros, handling NaN values
def safe_format_code(code, desired_length):
    if pd.isna(code):
        return np.nan  # Return NaN as is
    # Convert to int to remove any decimals, then format with leading zeros
    return '{:0>{width}}'.format(int(float(code)), width=desired_length)

# Convert fipsstatecode and fipscountycode to strings without decimals and ensure leading zeros
df_religion['fipsstatecode'] = df_religion['fipsstatecode'].apply(lambda x: safe_format_code(x, 2))
df_religion['fipscountycode'] = df_religion['fipscountycode'].apply(lambda x: safe_format_code(x, 3))

# Concatenate the cleaned state and county codes to create the FIPS code
df_religion['FIPS'] = df_religion['fipsstatecode'] + df_religion['fipscountycode']

# Optionally, drop the original state and county code columns if they are no longer needed
df_religion.drop(columns=['fipsstatecode', 'fipscountycode'], inplace=True)

df_religion.head(2)

,CBSA Code,Metro Name,2020 Population,Congregations,Adherents,"Congregations per 100,000 Population",Adherents as % of Population,Population Rank,Congregations Rank,Adherents Rank,...,metropolitandivisioncode,csacode,cbsatitle,metropolitanmicropolitanstatis,metropolitandivisiontitle,csatitle,countycountyequivalent,statename,centraloutlyingcounty,FIPS
0,10100,"Aberdeen, SD Micro Area",42287.0,101.0,31367.0,238.844089,0.741765,672.0,553.0,510.0,...,NaN,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,Brown County,South Dakota,Central,46013
1,10100,"Aberdeen, SD Micro Area",42287.0,101.0,31367.0,238.844089,0.741765,672.0,553.0,510.0,...,NaN,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,Edmunds County,South Dakota,Outlying,46045


In [19]:
# Rename the 'place' column to 'FIPS'  #7
df_transit.rename(columns={'place': 'FIPS'}, inplace=True)

# Remove double quotes from the 'FIPS' column values
df_transit['FIPS'] = df_transit['FIPS'].str.replace('"', '')

# Verify the changes
df_transit.head(2)

# FIPS is suspicious

,FIPS,name,blkgrps,population,households,alltransit_performance_score
0,0100100,"""Abanda""",0.03,40.96,15.98,0.0
1,0100124,"""Abbeville""",1.34,1282.40,476.84,0.0


In [20]:
state_fips_dict = {
    'Alabama': '01', 'Alaska': '02', 'Arizona': '04', 'Arkansas': '05',
    'California': '06', 'Colorado': '08', 'Connecticut': '09', 'Delaware': '10',
    'District of Columbia': '11', 'Florida': '12', 'Georgia': '13', 'Hawaii': '15',
    'Idaho': '16', 'Illinois': '17', 'Indiana': '18', 'Iowa': '19',
    'Kansas': '20', 'Kentucky': '21', 'Louisiana': '22', 'Maine': '23',
    'Maryland': '24', 'Massachusetts': '25', 'Michigan': '26', 'Minnesota': '27',
    'Mississippi': '28', 'Missouri': '29', 'Montana': '30', 'Nebraska': '31',
    'Nevada': '32', 'New Hampshire': '33', 'New Jersey': '34', 'New Mexico': '35',
    'New York': '36', 'North Carolina': '37', 'North Dakota': '38', 'Ohio': '39',
    'Oklahoma': '40', 'Oregon': '41', 'Pennsylvania': '42', 'Rhode Island': '44',
    'South Carolina': '45', 'South Dakota': '46', 'Tennessee': '47', 'Texas': '48',
    'Utah': '49', 'Vermont': '50', 'Virginia': '51', 'Washington': '53',
    'West Virginia': '54', 'Wisconsin': '55', 'Wyoming': '56'
}  #8

# Assuming df_marijuana is your DataFrame and it has a column named 'State'
# Map the state names to their FIPS codes
df_marijuana['STATEFIPS'] = df_marijuana['State'].map(state_fips_dict)

# Check the first few rows to verify the changes
df_marijuana.head(2)

,State,Legal Status,Medicinal,Decriminalized,STATEFIPS
0,Alabama,Mixed,Yes,No,01
1,Alaska,Fully Legal,Yes,Yes,02


In [21]:
# aggregate df_socioecon data at the FIPS level
# df_socioecon_2022.to_csv('df_socioecon_2022.csv', index=False)
# print(df_socioecon_2022.columns)
df_socioecon_2022 = df_socioecon_2022.drop(columns={'SAMPLE', 'SERIAL', 'CBSERIAL', 'HHWT', 'CLUSTER', 'STATEFIP', 'COUNTYFIP', 'STRATA', 'GQ',
                                'PERNUM', 'PERWT', 'RACE', 'RACED', 'EDUCD', 'EMPSTATD', 'YEAR'})

In [22]:
print(df_socioecon_2022.columns)
df_socioecon_2022.shape

Index(['CITYPOP', 'HHINCOME', 'VALUEH', 'SEX', 'AGE', 'RACHSING', 'EDUC',
       'EMPSTAT', 'IND', 'FIPS'],
      dtype='object')


(3373378, 10)

In [23]:
# values to columns
# ignore values not mentioned for qualitative columns
# CITYPOP: leave alone as only one value per FIPS
# HHINCOME: take average at FIPS level
# HHINCOME: take median at FIPS level for seperate columns
# VALUEH: take average at FIPS level
# SEX: 1 for SEX_Male; 2 for SEX_Female
# Age: take average at FIPS level
# Age: take median at FIPS level for seperate level
# RACHSING: 1 for RACE_White, 2 for RACE_Black, 3 for RACE_Native_American, 4 for RACE_AAPI, 5 for RACE_Hispanic
# EDUC: 06 for EDUC_HS, 07+08+09+10+11 for EDUC_College
# EMPSTAT: 1 for EMP_Employed, 2 for EMP_Unemployed, 3 for EMP_NotLaborForce
# IND: 0170-0490 for IND_Ag_For_Fish_Hunt_Mine, 770 for IND_Constr, 1070-3990 for IND_Manu, 4070-4590 for IND_Wholesale,
# 4670-5790 for IND_Retail, 6070-6390 + 0570-0690 for IND_Transpo_Ware_Util, 6470-6780 for IND_Info,
# 6870-7190 for IND_Fin_Ins_RealEstate, 8561-8690 for IND_Arts_Ent_Rec_Food, 8770-9290 for IND_OtherServices,
# 9370-9590 for IND_PublicAdmin, 9670-9870 for IND_Mil

In [24]:
# aggregate!

# Define a function for IND column categorization
def categorize_ind(ind):
    try:
        ind = int(ind)
        if 170 <= ind <= 490 or 570 <= ind <= 690:
            return 'IND_Ag_For_Fish_Hunt_Mine'
        elif ind == 770:
            return 'IND_Constr'
        elif 1070 <= ind <= 3990:
            return 'IND_Manu'
        elif 4070 <= ind <= 4590:
            return 'IND_Wholesale'
        elif 4670 <= ind <= 5790:
            return 'IND_Retail'
        elif 6070 <= ind <= 6390 or 570 <= ind <= 690:  # Adjusted for overlap
            return 'IND_Transpo_Ware_Util'
        elif 6470 <= ind <= 6780:
            return 'IND_Info'
        elif 6870 <= ind <= 7190:
            return 'IND_Fin_Ins_RealEstate'
        elif 8561 <= ind <= 8690:
            return 'IND_Arts_Ent_Rec_Food'
        elif 8770 <= ind <= 9290:
            return 'IND_OtherServices'
        elif 9370 <= ind <= 9590:
            return 'IND_PublicAdmin'
        elif 9670 <= ind <= 9870:
            return 'IND_Mil'
        else:
            return 'Other'
    except ValueError:
        return 'Other'

# Apply the IND categorization
df_socioecon_2022['IND_Cat'] = df_socioecon_2022['IND'].apply(categorize_ind)

# Recalculate EDUC_HS and EDUC_College
df_socioecon_2022['EDUC_HS'] = df_socioecon_2022['EDUC'].apply(lambda x: 1 if x == '6' else 0)
df_socioecon_2022['EDUC_College'] = df_socioecon_2022['EDUC'].isin(['7', '8', '9', '10', '11']).astype(int)

# Convert SEX, RACHSING, EMPSTAT, and IND_Cat to dummy variables
# Note: The drop_first=True parameter is used to avoid multicollinearity by dropping the first category.
qual_columns = ['SEX', 'RACHSING', 'EMPSTAT', 'IND_Cat']
df_socioecon_2022 = pd.get_dummies(df_socioecon_2022, columns=qual_columns, drop_first=True)

# Define aggregation rules
aggregation_rules = {
    'CITYPOP': 'max',  # Assuming max to pick the largest city's population in each FIPS
    'HHINCOME': ['mean', 'median'],
    'VALUEH': ['mean', 'median'],
    'AGE': ['mean', 'median'],
    'EDUC_HS': 'mean',  # Correctly capturing high school education
    'EDUC_College': 'mean',  # Correctly capturing college education
}

# Automatically include all dummy variables in the aggregation rules
for column in df_socioecon_2022.columns:
    if column.startswith(tuple([f"{x}_" for x in qual_columns])):  # Prefixes of dummy variables
        aggregation_rules[column] = 'mean'

# Apply the aggregation
df_agg = df_socioecon_2022.groupby('FIPS').agg(aggregation_rules).reset_index()

# Flatten MultiIndex columns if created by aggregation
df_agg.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_agg.columns.values]

df_agg.head()

,FIPS_,CITYPOP_max,HHINCOME_mean,HHINCOME_median,VALUEH_mean,VALUEH_median,AGE_mean,AGE_median,EDUC_HS_mean,EDUC_College_mean,...,IND_Cat_IND_Fin_Ins_RealEstate_mean,IND_Cat_IND_Info_mean,IND_Cat_IND_Manu_mean,IND_Cat_IND_Mil_mean,IND_Cat_IND_OtherServices_mean,IND_Cat_IND_PublicAdmin_mean,IND_Cat_IND_Retail_mean,IND_Cat_IND_Transpo_Ware_Util_mean,IND_Cat_IND_Wholesale_mean,IND_Cat_Other_mean
0,01000,1970,6.688749e+05,74805.0,2.807736e+06,270000.0,43.800921,45.0,0.0,0.0,...,0.029887,0.006704,0.074008,0.003706,0.028541,0.031988,0.066313,0.025354,0.011213,0.626700
1,01003,0,6.932327e+05,90000.0,2.359147e+06,345000.0,46.806620,51.0,0.0,0.0,...,0.038328,0.002323,0.045296,0.002904,0.026713,0.024390,0.070848,0.020906,0.011614,0.627758
2,01015,0,1.003894e+06,65750.0,3.222038e+06,250000.0,45.003976,48.0,0.0,0.0,...,0.021869,0.004970,0.075547,0.002982,0.039761,0.040755,0.064612,0.020875,0.011928,0.633201
3,01083,0,9.134908e+05,96100.0,2.548395e+06,300000.0,43.588101,46.0,0.0,0.0,...,0.016018,0.004577,0.090389,0.002288,0.038902,0.037757,0.037757,0.011442,0.006865,0.646453
4,01097,0,6.634464e+05,71000.0,3.109846e+06,275000.0,44.226492,46.0,0.0,0.0,...,0.028346,0.006814,0.076043,0.003543,0.030526,0.021259,0.068411,0.024530,0.015536,0.628782


In [25]:
df_agg.columns

Index(['FIPS_', 'CITYPOP_max', 'HHINCOME_mean', 'HHINCOME_median',
       'VALUEH_mean', 'VALUEH_median', 'AGE_mean', 'AGE_median',
       'EDUC_HS_mean', 'EDUC_College_mean', 'SEX_2_mean', 'EMPSTAT_1_mean',
       'EMPSTAT_2_mean', 'EMPSTAT_3_mean',
       'IND_Cat_IND_Arts_Ent_Rec_Food_mean', 'IND_Cat_IND_Constr_mean',
       'IND_Cat_IND_Fin_Ins_RealEstate_mean', 'IND_Cat_IND_Info_mean',
       'IND_Cat_IND_Manu_mean', 'IND_Cat_IND_Mil_mean',
       'IND_Cat_IND_OtherServices_mean', 'IND_Cat_IND_PublicAdmin_mean',
       'IND_Cat_IND_Retail_mean', 'IND_Cat_IND_Transpo_Ware_Util_mean',
       'IND_Cat_IND_Wholesale_mean', 'IND_Cat_Other_mean'],
      dtype='object')

In [26]:
# note: EDUC error persists!

In [28]:
# Ensure all FIPS codes are strings to work with them easily
df_land_2022['FIPS'] = df_land_2022['FIPS'].astype(str).str.zfill(5)
df_agg['FIPS'] = df_agg['FIPS_'].astype(str).str.zfill(5)
df_politics['FIPS'] = df_politics['county_fips'].astype(str).str.zfill(5)
df_religion['FIPS'] = df_religion['FIPS'].astype(str).str.zfill(5)
df_transit['FIPS'] = df_transit['FIPS'].astype(str).str.zfill(5)

# For df_marijuana, ensure STATEFIPS is a string for consistent merging
df_marijuana['STATEFIPS'] = df_marijuana['STATEFIPS'].astype(str).str.zfill(2)

# For dataframes with full FIPS codes, extract the STATEFIPS (first two digits) for merging with df_marijuana
df_land_2022['STATEFIPS'] = df_land_2022['FIPS'].str[:2]
df_agg['STATEFIPS'] = df_agg['FIPS'].str[:2]
df_politics['STATEFIPS'] = df_politics['FIPS'].str[:2]
df_religion['STATEFIPS'] = df_religion['FIPS'].str[:2]
df_transit['STATEFIPS'] = df_transit['FIPS'].str[:2]

# Start merging using 'outer' join to ensure all data is kept
merged_df = pd.merge(df_land_2022, df_agg, on='FIPS', how='outer', suffixes=('', '_agg'))
merged_df = pd.merge(merged_df, df_politics, on='FIPS', how='outer', suffixes=('', '_politics'))
merged_df = pd.merge(merged_df, df_religion, on='FIPS', how='outer', suffixes=('', '_religion'))
merged_df = pd.merge(merged_df, df_transit, on='FIPS', how='outer', suffixes=('', '_transit'))

# Before merging df_marijuana, ensure it's prepared to merge based on STATEFIPS
# Since df_marijuana uses STATEFIPS and does not have FIPS, it suggests a different merge strategy might be needed
# For demonstration, assuming STATEFIPS has been prepared in all dataframes and you're ready to merge based on it
merged_df = pd.merge(merged_df, df_marijuana, on='STATEFIPS', how='outer', suffixes=('', '_marijuana'))

In [29]:
merged_df.shape

(105153, 89)

In [30]:
merged_df.to_csv('Koob_Master.csv')